In [55]:
import pandas as pd
import gradio as gr
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Load and preprocess the dataset
def load_rules(file_path='Market_Basket_Optimisation_1.csv', min_support=0.002, min_lift=1.0):
    df = pd.read_csv(file_path)
    df['products'] = df['products'].astype(str).apply(lambda x: x.split(','))

    te = TransactionEncoder()
    te_ary = te.fit(df['products']).transform(df['products'])
    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

    frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=min_lift)

    rules['antecedents_str'] = rules['antecedents'].apply(lambda x: ', '.join(sorted(list(x))))
    rules['consequents_str'] = rules['consequents'].apply(lambda x: ', '.join(sorted(list(x))))

    return rules[['antecedents_str', 'consequents_str', 'confidence', 'lift']]

# Load rules once
rules_df = load_rules()

# Define recommendation function
def recommend_products(antecedents):
    input_cleaned = ', '.join(sorted([item.strip().lower() for item in antecedents.split(',')]))
    matching = rules_df[rules_df['antecedents_str'] == input_cleaned]

    if matching.empty:
        return "No matching bundles found."
    else:
        return matching.sort_values(by="lift", ascending=False).reset_index(drop=True)[['consequents_str', 'confidence', 'lift']]

# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 🛍️ Product Bundling Recommender")
    gr.Markdown("Enter products (comma-separated), e.g., `milk, bread`")

    input_text = gr.Textbox(label="Antecedents")
    output_table = gr.Dataframe(label="Recommended Consequents")

    btn = gr.Button("Recommend")
    btn.click(fn=recommend_products, inputs=input_text, outputs=output_table)

demo.launch()


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
